## **OBLIGATORIO - VISUALIZACIÓN DE DATOS CON PYTHON**
##Grupo 449. Docente: Joaquín González - Gustavo Juantorena.
### *Subgrupo A. Integrantes: Agustín Canavese, Sofía Vales.*

# Visualización de Datos con Python

## Proyecto Final: Creación de un Tablero de Datos Interactivo con Dash

El objetivo de este proyecto es desarrollar un tablero de datos interactivo utilizando la biblioteca [Dash](https://dash.plotly.com/) en Python. Para ello se pone a disposición un conjunto de datos histórico sobre los Juegos Olímpicos modernos para construir visualizaciones significativas y permitir interacciones con el usuario.


## # 1. Data Loading and Dataset Preparation

In [1]:
# Install required packages
%%capture
!pip install plotly dash

In [2]:
%%capture
!pip install dash-bootstrap-components plotly dash jupyter-dash

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output

In [5]:
# Load dataframe
df = pd.read_csv("juegos_olimpicos.csv")

df.drop_duplicates(subset=None, keep="first", inplace =True)

# Visualization section

In [17]:
# Define layout style
stylesheet = [dbc.themes.BOOTSTRAP]

# Define the Dash app
app = Dash(__name__, external_stylesheets=stylesheet, suppress_callback_exceptions=True)

# Define application layout
app.layout = html.Div([html.H1("Tablero Interactivo de los Juegos Olímpicos (1896-2016)"),
    dcc.Tabs(id="tabs", value="tab_1", children=[
        dcc.Tab(label="Conteo de Medallas", value="tab_1"),
        dcc.Tab(label="Participación por Sexo", value="tab_2"),
        dcc.Tab(label="Representación Geográfica", value="tab_3"),
        dcc.Tab(label="Distribución de Edad y Sexo", value="tab_4")]),
    html.Div(id="tabs-content")])

# Render generation
@app.callback(Output("tabs-content", "children"),
              [Input("tabs", "value")])
def render_content(tab):

    if tab == "tab_1":
      return html.Div([
          html.H3("Conteo de Medallas"),
          html.H6("Selector de cantidad de países"),
          dcc.Dropdown(id="k_selector_ej_1", placeholder="Seleccionar cantidad de países a mostrar"),
          html.Br(),
          dcc.Checklist(id="medal_selector_ej_1",options=[
              {"label": html.Div(['Gold'], style={'color': 'Gold', 'font-size': 20}), "value": "Gold"},
               {"label": html.Div(['Silver'], style={'color': 'Silver', 'font-size': 20}), "value": "Silver"},
                {"label": html.Div(['Bronze'], style={'color': 'brown', 'font-size': 20}), "value": "Bronze"},],
                        value=["Gold", "Silver", "Bronze"], labelStyle={"display": "flex", "align-items": "center"}),
          html.Br(),
          html.H6("Selector de disciplina"),
          dcc.Dropdown(id="sport_selector_ej_1", placeholder="Seleccionar la disciplina a mostrar"),
          html.Br(),
          dcc.RangeSlider(id="year_slider_ej_1", min=df["Year"].min(), max=df["Year"].max(), step=1, value=[df["Year"].min(), df["Year"].max()],
                          allowCross=False, marks={str(year): str(year) for year in df["Year"].unique()},),
          html.Br(),
          dcc.Graph(id="fig_ej_1")])

    elif tab == "tab_2":
        return html.Div([
            html.H3("Participación por Sexo"),
            dcc.RangeSlider(id="year_slider_ej_2", min=df["Year"].min(), max=df["Year"].max(), step=1,
                            value=[df["Year"].min(), df["Year"].max()], marks={str(year): str(year) for year in df["Year"].unique()},),
            dcc.Graph(id="fig_ej_2")])

    elif tab == "tab_3":
          return html.Div([
              html.H3("Representación Geográfica"),
              dcc.Dropdown(id="year_selector_ej_3", options=[], value=None),
              dcc.Graph(id="fig_ej_3")])

    elif tab == "tab_4":
      return dbc.Container(
    html.Div([
        html.H3("Distribución de Edad por Sexo y Disciplina"),

        # Control panel container
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        html.H6("Selección de disciplina"),
                        dcc.Dropdown(id="sport_selector_ej_4", options=[], placeholder="Seleccionar disciplina a mostrar"),
                        html.Br(),
                        html.H6("Rango de Olimpiadas"),
                        dcc.RangeSlider(id="year_slider_ej_4", min=df["Year"].min(), max=df["Year"].max(), step=1,
                                        value=[df["Year"].min(), df["Year"].max()], allowCross=False,
                                        marks={str(year): str(int(year)) for year in df["Year"].unique()}),]),
                    style={"padding": "20px", "border": "2px solid black", "border-radius": "10px"}), width=12), justify="center", style={"margin-bottom": "20px"}),

        # Charts and age-filter container
        dbc.Row([
            dbc.Col(dcc.Graph(id="fig_ej_4"), width=8),  # Histograma
            dbc.Col(
                html.Div([
                    dbc.Card(
                        dbc.CardBody([
                            html.H6("Seleccionar Edad para el Gráfico de Torta"),
                            dcc.Dropdown(id="age_dropdown_ej_4", placeholder="Seleccionar edad", style={"width": "100%"})]),
                        color="light", outline=True, style={"padding": "20px", "border": "2px solid black", "border-radius": "10px"}),
                    dcc.Graph(id="pie_chart_ej_4")  # Gráfico de torta
                ]), width=4)], justify="center", align="center")]),
    fluid=True)



# Callback definition for tab_1
@app.callback([Output("k_selector_ej_1", "options"),
               Output("sport_selector_ej_1", "options"),
               Output("fig_ej_1", "figure")],
                [Input("k_selector_ej_1", "value"),
                 Input("sport_selector_ej_1", "value"),
                 Input("medal_selector_ej_1", "value"),
                 Input("year_slider_ej_1", "value")])

def update_ejercicio_1(selected_k, selected_sport, selected_medals, selected_year_range):

    # Filtering by year range and medal category
    df_filtered = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]
    df_filtered = df_filtered[df_filtered["Medal"].isin(selected_medals)]

    # Number of countries selection options
    k_options = [{"label": str(i), "value": i} for i in range(1, len(df_filtered["NOC"].unique()) + 1)]

    # Sport discipline selection options
    sport_options = [{"label": "all sports", "value": "all sports"}] + \
                    [{"label": sport, "value": sport} for sport in sorted(df_filtered["Sport"].unique())]

    # Validation of input parameters
    if not selected_k or not selected_sport:
      fig = px.bar(title="Por favor, seleccione todas las opciones", labels={"x": "Medalla", "y": "Número de Medallas"})
      return k_options, sport_options, fig

    # Additional filtering by sport
    df_filtered_ej_1 = df_filtered[df_filtered["Sport"] == selected_sport] if selected_sport != "all sports" else df_filtered

    # Data validation after filtering
    if len(df_filtered_ej_1) == 0:
      fig = px.bar(title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
      return k_options, sport_options, fig

    # Grouping by country and medal type
    df_grouped = df_filtered_ej_1.groupby(["NOC", "Medal"]).size().unstack(fill_value=0).reset_index()

    # Data validation after grouping
    if len(df_grouped) == 0:
        fig = px.bar(title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
        return k_options, sport_options, fig

    # Check available medal categories
    available_medals = [medal for medal in selected_medals if medal in df_grouped.columns]
    if not available_medals:
        fig = px.bar(title="No hay medallas disponibles para los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
        return k_options, sport_options, fig

    # Sort and select top-k countries
    df_grouped["total_medals"] = df_grouped[available_medals].sum(axis=1)
    df_grouped = df_grouped.sort_values(by="total_medals", ascending=False)
    selected_k = min(selected_k, len(df_grouped))
    df_top_paises = df_grouped.head(selected_k)

    # Transform DataFrame to long format
    df_long = pd.melt(df_top_paises, id_vars=['NOC'], value_vars=available_medals, var_name='Medal', value_name='Number of Medals')

    # Generate the Plotly figure
    fig = px.bar(df_long, x='Number of Medals', y='NOC', color='Medal', barmode='group', color_discrete_map={"Gold": "gold", "Silver": "silver", "Bronze": "brown"},
                 category_orders={"NOC": df_top_paises['NOC'].tolist()},
                 title=f"Top {selected_k} países con más medallas en {selected_sport if selected_sport != 'all sports' else 'todos los deportes'}")
    fig.update_layout(xaxis_title="Número de Medallas", yaxis_title="Países")

    return k_options, sport_options, fig


# Callback definition for tab_2
@app.callback(Output("fig_ej_2", "figure"),
 [Input("year_slider_ej_2", "value")])

def update_ejercicio_2(selected_year_range):
    # Filter the dataset by the selected year range
    df_anios = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]

    # Group and count the number of male and female athletes by country and year
    df_agrupado = df_anios.groupby(["NOC", "Year", "Sex"]).size().reset_index(name="count")

    # Pivot the table to create separate columns for male and female counts
    df_sexo = df_agrupado.pivot(index=["NOC", "Year"], columns="Sex", values="count").fillna(0).reset_index()

    # Convert the "Year" column to string to treat it as a categorical variable and improve visualisation
    df_sexo["Year"] = df_sexo["Year"].astype(str)

    # Create a scatter plot using Plotly Express, assigning a different color to each year
    fig = px.scatter(df_sexo, x="M", y="F", color="Year", labels={"M": "Cantidad de Hombres", "F": "Cantidad de Mujeres"},
                     title="Participación por Sexo en los Juegos Olímpicos, según país y año",
                     color_discrete_sequence=px.colors.qualitative.Bold)

    # Add linear regression lines for each year with a gradient color scale
    years = df_sexo["Year"].unique()
    color_scale = px.colors.sequential.Viridis

    for i, year in enumerate(years):
        df_year = df_sexo[df_sexo["Year"] == year]
        x = df_year["M"]
        y = df_year["F"]

    # Compute the regression using a linear function
        coef = np.polyfit(x, y, 1)
        poly1d_fn = np.poly1d(coef)

    # Add the regression line to the figure using go.Scatter
        fig.add_trace(go.Scatter(x=x, y=poly1d_fn(x), mode='lines', name=f'Regresión {year}'))

    return fig


# Callback definition for tab_3
@app.callback(
    [Output("year_selector_ej_3", "options"), Output("year_selector_ej_3", "value"), Output("fig_ej_3", "figure")],
    [Input("year_selector_ej_3", "value")]
)
def update_ejercicio_3(selected_year):
    # Extract and sort unique year values
    years = sorted(df['Year'].unique())
    options = [{"label": str(year), "value": year} for year in years]

    # Define a default year if none is selected
    if not selected_year or selected_year not in years:
        selected_year = years[0]

    # Filter the dataset according to the selected year
    df_anios_2 = df[df["Year"] == selected_year]

    # Aggregate and count athletes by country
    df_atletas = df_anios_2.groupby("NOC").size().reset_index(name="count")

    # Generate a choropleth world map to visualise athlete distribution by country using Plotly
    fig = px.choropleth(
        df_atletas,
        locations="NOC",
        color="count",  # Selected variable for visualisation
        hover_name="NOC",  # Show country code on hover interaction
        color_continuous_scale="Purples",
        title=f"Distribución Geográfica de Atletas en los Juegos Olímpicos en {selected_year}"
    )

    return options, selected_year, fig


# Callback definition for tab_4
@app.callback([Output("sport_selector_ej_4", "options"),
               Output("age_dropdown_ej_4", "options"),
               Output("fig_ej_4", "figure"),
               Output("pie_chart_ej_4", "figure")],
                [Input("year_slider_ej_4", "value"),
                 Input("sport_selector_ej_4", "value"),
                 Input("age_dropdown_ej_4", "value")])

def update_ejercicio_4(selected_year_range, selected_sport, selected_age):

    # Extract and alphabetically sort unique disciplines
    sports = sorted(df["Sport"].unique())
    lista_sports = [{"label": "all sports", "value": "all sports"}] + [{"label": sport, "value": sport} for sport in sports]

    # Filter dataset based on selected year range and sport
    df_filtered_ej_4 = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]

    if selected_sport != "all sports":
        df_filtered_ej_4 = df_filtered_ej_4[df_filtered_ej_4["Sport"] == selected_sport]

    # Update age range dropdown options
    edades_disponibles = sorted(df_filtered_ej_4["Age"].dropna().unique(), reverse=True)
    lista_edad = [{"label": str(int(age)), "value": age} for age in edades_disponibles]

    # Update histogram visualization
    if not selected_year_range or len(selected_year_range) != 2:
        fig_histogram = px.histogram(x=[], title="Rango de olimpiadas seleccionado no válido", labels={"x": "Edad"})
        fig_histogram.update_layout(xaxis_title="Edad", yaxis_title="Número de Atletas", title=dict(text="Rango de olimpiadas seleccionado no válido", font=dict(size=18)))

        fig_pie = px.pie(title="Seleccione una edad válida") # Initialize empty pie chart

        return lista_sports, lista_edad, fig_histogram, fig_pie

    if len(df_filtered_ej_4) == 0:
      fig_histogram = px.histogram(x=[], title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Edad"})
      fig_histogram.update_layout(xaxis_title="Edad", yaxis_title="Número de Atletas", title=dict(text="No hay datos para mostrar con los filtros seleccionados",
                                                                                                  font=dict(size=18)))

      fig_pie = px.pie(title="No hay datos para mostrar con los filtros seleccionados") # Initialize empty pie chart

      return lista_sports, lista_edad, fig_histogram, fig_pie

    # Modify gender labels
    df_filtered_ej_4["Sex"] = df_filtered_ej_4["Sex"].replace({"F": "Mujeres", "M": "Hombres"})

    # Generate the histogram visualization
    fig_histogram = px.histogram(df_filtered_ej_4, x="Age", color="Sex", nbins=30,
                                 title=f"Distribución de Edad por Sexo y Disciplina: {selected_sport}",
                                 labels={"Age": "Edad"}, color_discrete_map={"Mujeres": "#FF97FF", "Hombres": "#19D3F3"})
    fig_histogram.update_layout(barmode="overlay", xaxis_title="Edad", yaxis_title="Número de Atletas", plot_bgcolor='rgba(0, 0, 0, 0)',
                                title=dict(text=f"Distribución de Edad por Sexo y Disciplina: {selected_sport}", font=dict(size=18)))
    fig_histogram.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig_histogram.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig_histogram.update_traces(opacity=0.6)

    # For the pie chart, filter data by the selected age and aggregate by gender
    df_pie = df_filtered_ej_4[df_filtered_ej_4["Age"] == selected_age]

    if len(df_pie) == 0:
        fig_pie = px.pie(title="No hay datos para la edad seleccionada")
    else:
        df_pie = df_pie.groupby("Sex").size().reset_index(name="Count")

        fig_pie = px.pie(df_pie, names="Sex", values="Count", title=f"Distribución por Género para la Edad {selected_age}",
                         labels={"Sex": "Sexo", "Count": "Número de Atletas"}, color="Sex", color_discrete_map={"Mujeres": "rgb(253,205,172)", "Hombres": "rgb(56,166,165)"})
        fig_pie.update_traces(textinfo="percent+label")
        fig_pie.update_layout(title=dict(text=f"Distribución por Género para el rango <br> de olimpiadas seleccionado y edad de {selected_age}",
                                         font=dict(size=16), x=0.5, xanchor='center', y=0.95, yanchor='top'), title_font_size=16, margin=dict(t=80))

    return lista_sports, lista_edad, fig_histogram, fig_pie


# ---- RUN DASH INSIDE COLAB (INLINE) ----
import threading
import time
from google.colab import output

PORT = 8050

def run_app():
    # use_reloader=False
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

time.sleep(1)
output.serve_kernel_port_as_iframe(PORT, width=1200, height=800)

Dash is running on http://0.0.0.0:8050/



INFO:dash.dash:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


<IPython.core.display.Javascript object>